In [46]:
import pandas as pd
import re
from bs4 import BeautifulSoup

In [47]:
def has_environmental_claim(text):
    """
    Detect actual environmental claims, not just keywords
    """
    if pd.isna(text) or len(str(text)) < 20:
        return False
    
    text_lower = str(text).lower()
    
    # Stage 1: Must have environmental context words
    environmental_phrases = [
        # Explicit environmental claims
        'eco-friendly', 'eco friendly', 'environmentally friendly',
        'sustainable', 'sustainability',
        'organic', 'certified organic',
        'recycled', 'recyclable',
        'biodegradable', 'compostable',
        'carbon neutral', 'carbon footprint', 'carbon offset',
        'zero waste', 'plastic-free', 'plastic free',
        'fair trade',
        'green product', 'green living',
        'earth friendly', 'earth-friendly',
        'planet friendly', 'planet-friendly',
        
        # Certifications (strong signal)
        'gots', 'fsc certified', 'usda organic', 'grs certified',
        'ecocert', 'oeko-tex', 'fair trade certified',
        
        # Materials with environmental context
        'organic cotton', 'recycled polyester', 'recycled plastic',
        'bamboo fiber', 'hemp fiber',
        'post-consumer recycled', 'pre-consumer recycled'
    ]
    
    # Check if any phrase exists
    return any(phrase in text_lower for phrase in environmental_phrases)

In [48]:
# Read in chunks (for large files)
chunk_size = 50000
filtered_data = []

print("Applying stricter environmental claim filter...")

Applying stricter environmental claim filter...


In [49]:
for i, chunk in enumerate(pd.read_csv('amazon_products.csv', chunksize=chunk_size)):
    # Combine text fields
    chunk['combined_text'] = (
        chunk['TITLE'].fillna('') + ' ' +
        chunk['DESCRIPTION'].fillna('')
    )
    
    # Apply stricter filter
    env_mask = chunk['combined_text'].apply(has_environmental_claim)
    filtered = chunk[env_mask]
    
    if len(filtered) > 0:
        filtered_data.append(filtered)
    
    print(f"Chunk {i+1}: Found {len(filtered)} environmental products")

Chunk 1: Found 906 environmental products
Chunk 2: Found 890 environmental products
Chunk 3: Found 934 environmental products
Chunk 4: Found 914 environmental products
Chunk 5: Found 871 environmental products
Chunk 6: Found 912 environmental products
Chunk 7: Found 889 environmental products
Chunk 8: Found 925 environmental products
Chunk 9: Found 924 environmental products
Chunk 10: Found 869 environmental products
Chunk 11: Found 908 environmental products
Chunk 12: Found 864 environmental products
Chunk 13: Found 894 environmental products
Chunk 14: Found 926 environmental products
Chunk 15: Found 855 environmental products
Chunk 16: Found 949 environmental products
Chunk 17: Found 886 environmental products
Chunk 18: Found 935 environmental products
Chunk 19: Found 914 environmental products
Chunk 20: Found 902 environmental products
Chunk 21: Found 926 environmental products
Chunk 22: Found 979 environmental products
Chunk 23: Found 897 environmental products
Chunk 24: Found 903 

In [50]:
df_env = pd.concat(filtered_data, ignore_index=True)
print(f"\nTotal after stricter filter: {len(df_env)} products")


Total after stricter filter: 40658 products


In [51]:
def is_false_positive(text):
    """
    Detect false positives (not actual environmental claims)
    """
    if pd.isna(text):
        return True
    
    text_lower = str(text).lower()
    
    # False positive patterns
    false_patterns = [
        # Color descriptions
        r'\bgreen\s+(color|shirt|dress|paint|dye)\b',
        r'\bnatural\s+(color|shade|tone)\b',
        
        # Features/modes (not claims)
        r'\beco\s+mode\b',
        r'\bgreen\s+screen\b',
        r'\bnatural\s+light\b',
        
        # Wood/material descriptions (not claims)
        r'\bnatural\s+wood\b',
        r'\bnatural\s+stone\b',
        
        # Too short (likely just keywords)
        r'^.{0,30}$'  # Less than 30 characters
    ]
    
    # Check for false patterns
    for pattern in false_patterns:
        if re.search(pattern, text_lower):
            return True
    
    return False

In [52]:
# Remove false positives
df_env = df_env[~df_env['combined_text'].apply(is_false_positive)]
print(f"After removing false positives: {len(df_env)} products")

After removing false positives: 40089 products


In [53]:
# Remove very short texts (likely incomplete data)
df_env['text_length'] = df_env['combined_text'].str.len()
df_env = df_env[df_env['text_length'] >= 50]  # At least 50 characters

print(f"After length filter: {len(df_env)} products")

After length filter: 39708 products


In [54]:
# Remove duplicates
df_env = df_env.drop_duplicates(subset=['combined_text'], keep='first')
print(f"After removing duplicates: {len(df_env)} products")

After removing duplicates: 39487 products


In [55]:
def categorize_claim_strength(text):
    """
    Categorize how strong the environmental claim is
    """
    text_lower = str(text).lower()
    
    # Strong claims (have certifications or metrics)
    strong_indicators = ['certified', 'gots', 'fsc', 'usda', 'fair trade', r'\d+%']
    if any(re.search(ind, text_lower) for ind in strong_indicators):
        return 'strong'
    
    # Medium claims (specific materials/processes)
    medium_indicators = ['organic cotton', 'recycled polyester', 'bamboo fiber', 
                        'post-consumer', 'carbon neutral']
    if any(ind in text_lower for ind in medium_indicators):
        return 'medium'
    
    # Weak claims (just buzzwords)
    return 'weak'


In [56]:
# Categorize all claims
df_env['claim_strength'] = df_env['combined_text'].apply(categorize_claim_strength)

print("\nClaim strength distribution:")
print(df_env['claim_strength'].value_counts())

# Stratified sampling: Get balanced mix
sample_sizes = {
    'strong': 1500,   # More strong claims (easier to label as SPECIFIC)
    'medium': 1500,   # Medium claims (mixed)
    'weak': 1000      # Some weak claims (likely VAGUE)
}



Claim strength distribution:
claim_strength
weak      24941
strong    13569
medium      977
Name: count, dtype: int64


In [57]:
# More strong/medium, fewer weak
sample_config = {
    'strong': 2000,   # Plenty available (13,569)
    'medium': 977,    # Take all (only 977 available)
    'weak': 1000      # Reduced from 24,941
}

samples = []
for strength, target_n in sample_config.items():
    subset = df_env[df_env['claim_strength'] == strength]
    n = min(len(subset), target_n)
    sample = subset.sample(n=n, random_state=42)
    samples.append(sample)
    print(f"{strength}: {n}")

df_sample = pd.concat(samples, ignore_index=True)
print(f"\nTotal: {len(df_sample)}")

strong: 2000
medium: 977
weak: 1000

Total: 3977


In [58]:
def clean_html_text(text):
    """
    Remove HTML tags and clean text
    """
    if pd.isna(text):
        return ""
    
    # Remove HTML tags using BeautifulSoup
    soup = BeautifulSoup(str(text), 'html.parser')
    text_clean = soup.get_text()
    
    # Clean up extra whitespace
    text_clean = re.sub(r'\s+', ' ', text_clean)
    text_clean = text_clean.strip()
    
    return text_clean

In [59]:
print(f"\nTotal sampled: {len(df_sample):,}")
print("\nNew distribution:")
print(df_sample['claim_strength'].value_counts())


Total sampled: 3,977

New distribution:
claim_strength
strong    2000
weak      1000
medium     977
Name: count, dtype: int64


In [60]:
df_sample.columns.tolist()
df_sample.head()

,PRODUCT_ID,TITLE,BULLET_POINTS,DESCRIPTION,PRODUCT_TYPE_ID,PRODUCT_LENGTH,combined_text,text_length,claim_strength
0,1064258,L'ovedbaby Unisex-Baby Organic Cotton Footed O...,[Infant footie with snap buttons from neck to ...,"OR444s-0/3M Size: 0 - 3 Months, Color: Sage Fe...",2364,100.000000,L'ovedbaby Unisex-Baby Organic Cotton Footed O...,464,strong
1,2171982,K3 Deluxe Virgin Plastic Bucket (Balti) 25 (23...,[K3 Deluxe Virgin Plastic Bucket (Balti) 25 li...,<b>Sturdy and Light Weight Made from 100% virg...,424,1614.173227,K3 Deluxe Virgin Plastic Bucket (Balti) 25 (23...,1370,strong
2,2400935,"40-Piece Flatware Set For 8, EIUBUIE 18/10 Sta...",[👍【FINEST STAINLESS STEEL】18/10 stainless-stee...,<b>EIUBUIE 40 Piece High Quality 18/10 Stainle...,8184,1043.000000,"40-Piece Flatware Set For 8, EIUBUIE 18/10 Sta...",2160,strong
3,2924934,TUFFPAULIN 15FT X 15FT 120 GSM Silver Tarpauli...,[Actual size will be approx. half foot shorter...,TUFFPAULIN tarpaulin are made from superior qu...,10249,18000.000000,TUFFPAULIN 15FT X 15FT 120 GSM Silver Tarpauli...,1801,strong
4,2518472,"Shrayati Areca Leaf Round Bowls, 5 Inch, Pack ...",[Biodegradability - Every small step towards t...,Shrayati Areca Leaf Round bowls are made from ...,1426,492.125984,"Shrayati Areca Leaf Round Bowls, 5 Inch, Pack ...",627,strong


In [61]:
# Clean text
df_sample['combined_text'] = df_sample['combined_text'].apply(clean_html_text)
df_sample['combined_text'] = df_sample['combined_text'].str.replace(r'\s+', ' ', regex=True)
df_sample['combined_text'] = df_sample['combined_text'].str.strip()

In [62]:
# AUTO-LABEL
print("\n" + "="*50)
print("AUTO-LABELING")
print("="*50)

def auto_label(text, strength):
    """
    Auto-label with claim strength as hint
    """
    if pd.isna(text) or len(str(text)) < 20:
        return 'SKIP'
    
    text_lower = str(text).lower()
    
    # SPECIFIC indicators
    certifications = [
        'certified', 'certification',
        'gots', 'fsc', 'usda organic', 'grs', 
        'fair trade', 'fairtrade',
        'ecocert', 'oeko-tex',
        'rainforest alliance',
        'certified by', 'approved by', 'verified by'
    ]
    has_cert = any(cert in text_lower for cert in certifications)
    
    # Has percentages or measurements
    has_metrics = bool(re.search(r'\d+\s*%|\d+\s*grams?|\d+\s*kg', text))
    
    # Specific materials/processes with details
    specific_details = [
        'organic cotton', 'recycled polyester', 'recycled plastic',
        'post-consumer recycled', 'pre-consumer recycled',
        'carbon neutral', 'carbon offset', 'carbon footprint'
    ]
    has_specific = any(detail in text_lower for detail in specific_details)
    
    # VAGUE indicators (just buzzwords)
    vague_only = [
        'eco-friendly', 'eco friendly',
        'environmentally friendly',
        'sustainable', 'sustainability',
        'green', 'green product',
        'natural', 'nature',
        'earth friendly', 'planet friendly'
    ]
    has_vague_only = any(term in text_lower for term in vague_only)
    
    # DECISION LOGIC with strength hints
    
    # Strong claims: likely SPECIFIC
    if strength == 'strong':
        if has_cert or has_metrics:
            return 'SPECIFIC'
        elif has_specific:
            return 'SPECIFIC'
        else:
            return 'UNCERTAIN'
    
    # Weak claims: likely VAGUE
    elif strength == 'weak':
        if has_cert or has_metrics:
            return 'SPECIFIC'  # Even weak claims can have certs
        elif has_vague_only and not has_specific:
            return 'VAGUE'
        else:
            return 'UNCERTAIN'
    
    # Medium claims: mixed
    else:  # medium
        if has_cert or has_metrics:
            return 'SPECIFIC'
        elif has_specific:
            return 'SPECIFIC'
        elif has_vague_only:
            return 'VAGUE'
        else:
            return 'UNCERTAIN'


AUTO-LABELING


In [63]:
# Apply labeling
df_sample['label'] = df_sample.apply(
    lambda row: auto_label(row['combined_text'], row['claim_strength']),
    axis=1
)

print("\nLabel distribution (including UNCERTAIN):")
print(df_sample['label'].value_counts())


Label distribution (including UNCERTAIN):
label
SPECIFIC     2850
VAGUE         803
UNCERTAIN     324
Name: count, dtype: int64


In [64]:
# Keep only confident labels
df_labeled = df_sample[df_sample['label'].isin(['SPECIFIC', 'VAGUE'])].copy()

print(f"\nConfident labels: {len(df_labeled):,}")
print("\nFinal label distribution:")
print(df_labeled['label'].value_counts())


Confident labels: 3,653

Final label distribution:
label
SPECIFIC    2850
VAGUE        803
Name: count, dtype: int64


In [65]:
#Show label breakdown by strength
print("\nLabel breakdown by claim strength:")
print(pd.crosstab(df_labeled['claim_strength'], df_labeled['label']))


Label breakdown by claim strength:
label           SPECIFIC  VAGUE
claim_strength                 
medium               823     92
strong              1997      0
weak                  30    711


In [66]:
# Save training data
df_labeled[['combined_text', 'label']].to_csv('data/training_data.csv', index=False)
print(f"\n✓ Saved {len(df_labeled):,} labeled examples to 'training_data.csv'")


✓ Saved 3,653 labeled examples to 'training_data.csv'


In [68]:
# Show some examples
print("\n" + "="*50)
print("SAMPLE EXAMPLES")
print("="*50)

print("\nSTRONG claims labeled as SPECIFIC:")
strong_specific = df_labeled[
    (df_labeled['claim_strength'] == 'strong') & 
    (df_labeled['label'] == 'SPECIFIC')
].sample(3, random_state=42)
for text in strong_specific['combined_text']:
    print(f"- {text[:250]}...")

print("\nWEAK claims labeled as VAGUE:")
weak_vague = df_labeled[
    (df_labeled['claim_strength'] == 'weak') & 
    (df_labeled['label'] == 'VAGUE')
].sample(3, random_state=42)
for text in weak_vague['combined_text']:
    print(f"- {text[:250]}...")


SAMPLE EXAMPLES

STRONG claims labeled as SPECIFIC:
- Vitamin E Lip Balm by Real Purity Linden flowers, often used for treating colds and their related symptoms, soothe the lips and induce a calming sensation.;Chamomile acts as a non-allergenic anti-inflammatory that calms stressed and cracked lips.;Hyp...
- PVC COMPACT BALL VALVE 1/2" - Socket - Sanipro Plastic Inline Compact Ball Valve - 1-Piece Valve Structure - Made of Corrosion Resistant uPVC (unplasticized polyvinyl chloride) AKA rigid PVC, or vinyl siding - Rigid and durable; does not flex; fire-r...
- ArtzFolio Gold Silver Feathers Bulletin Board Notice Pin Board | Vision Soft Board Combo with Thumb Push Pins | White Frame 36 x 18 inch (91 x 46 cms) Pin Board; White Frame || SIZE: 36 x 18 inch (91 x 46 cms): including 0.75 inch wide White Frame HI...

WEAK claims labeled as VAGUE:
- Goshey Premium Yoga Mat for Gym Workout and Yoga Exercise, Anti-Slip/Skid Yoga Mat for Men & Women Fitness (White, 6 ft*2ft) Our Brand, Goshey pre